# 영업일 캘린더 (KRX)

* 한국거래소 홈페이지에서 휴장일을 xls로 받을 수 있다
* 휴장일 데이터로 부터 개장일(market days)를 생성
* 해외 시장(CME, EUREX)등은  pandas-market-calendars를 활용

<img width="320" src="http://i.imgur.com/ClIQff8.jpg" >

#### 2017 http://fb.com/financedata , http://financedata.github.io
<!-- TEASER_END -->

# 한국거래소 홈페이지 휴장일 조회
한국거래소 홈페이지 http://marketdata.krx.co.kr 에서 아래 메뉴에서 휴장일을 조회할 수 있다

#### 시장정보 > 시장동향 > 공시 > 증시일정 > 휴장일

링크는 아래와 같다

http://marketdata.krx.co.kr/contents/MKD/01/0110/01100305/MKD01100305.jsp

<img width="80%" src="http://i.imgur.com/xUGXh8f.png" >

한국거래소, CME, EUREX 캘린더를 년도별로 제공하고 있으며, '다운로드'를 누르면 엑셀(data.xls)파일로 다운로드 할 수 있다.

다운로드한 파일명(data.xls)을 적절하게 변경해준다. 여기서는 hdays-2017-KRX.xls 로 변경하고, 아래와 같이 github 에 공유파일 링크를 만들었다.

* hdays-2017-KRX.xls ( https://goo.gl/G1gzHh )

In [1]:
import pandas as pd

df_hdays = pd.read_excel('https://goo.gl/G1gzHh')
df_hdays

,일자 및 요일,요일구분
0,2017-01-27 (FRI),금요일
1,2017-01-30 (MON),월요일
2,2017-03-01 (WED),수요일
3,2017-05-01 (MON),월요일
4,2017-05-03 (WED),수요일
5,2017-05-05 (FRI),금요일
6,2017-05-09 (TUE),화요일
7,2017-06-06 (TUE),화요일
8,2017-08-15 (TUE),화요일
9,2017-10-03 (TUE),화요일


날짜 데이터에 요일이 함께 들어있어 가공이 필요하다.


In [2]:
hdays = df_hdays['일자 및 요일'].str.extract('(\d{4}-\d{2}-\d{2})', expand=False)
hdays = pd.to_datetime(hdays)
hdays.name = '날짜'
hdays

0    2017-01-27
1    2017-01-30
2    2017-03-01
3    2017-05-01
4    2017-05-03
5    2017-05-05
6    2017-05-09
7    2017-06-06
8    2017-08-15
9    2017-10-03
10   2017-10-04
11   2017-10-05
12   2017-10-06
13   2017-10-09
14   2017-12-25
15   2017-12-29
Name: 날짜, dtype: datetime64[ns]

# 개장일(market day)

2017년 중에 개장일(market day)목록을 만들어 보자. 우선 pandas.date_range() 함수를 사용하여 1월 1일~ 12월 31까지 날짜를 생성한다. (365일)

In [3]:
mdays = pd.date_range('2017-01-01', '2017-12-31')
mdays

DatetimeIndex(['2017-01-01', '2017-01-02', '2017-01-03', '2017-01-04',
               '2017-01-05', '2017-01-06', '2017-01-07', '2017-01-08',
               '2017-01-09', '2017-01-10',
               ...
               '2017-12-22', '2017-12-23', '2017-12-24', '2017-12-25',
               '2017-12-26', '2017-12-27', '2017-12-28', '2017-12-29',
               '2017-12-30', '2017-12-31'],
              dtype='datetime64[ns]', length=365, freq='D')

이때 freq 를 'B'로 지정(business day)하면, 주말(토,일)은 제외된다. (260일)

날짜 생성에 대해서는 다음 URL을 참고 하자. 
https://pandas.pydata.org/pandas-docs/stable/timeseries.html

특히, 간격 별칭(Offset Aliases)에 대해서는 아래 표에 정리되어 있다.

https://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases

In [4]:
mdays = pd.date_range('2017-01-01', '2017-12-31', freq='B')
mdays

DatetimeIndex(['2017-01-02', '2017-01-03', '2017-01-04', '2017-01-05',
               '2017-01-06', '2017-01-09', '2017-01-10', '2017-01-11',
               '2017-01-12', '2017-01-13',
               ...
               '2017-12-18', '2017-12-19', '2017-12-20', '2017-12-21',
               '2017-12-22', '2017-12-25', '2017-12-26', '2017-12-27',
               '2017-12-28', '2017-12-29'],
              dtype='datetime64[ns]', length=260, freq='B')

여기서 휴일을 제외(df.drop)하면, 모든 영업일(market day)가 만들어진다. (244일)

In [5]:
mdays = mdays.drop(hdays)
mdays

DatetimeIndex(['2017-01-02', '2017-01-03', '2017-01-04', '2017-01-05',
               '2017-01-06', '2017-01-09', '2017-01-10', '2017-01-11',
               '2017-01-12', '2017-01-13',
               ...
               '2017-12-14', '2017-12-15', '2017-12-18', '2017-12-19',
               '2017-12-20', '2017-12-21', '2017-12-22', '2017-12-26',
               '2017-12-27', '2017-12-28'],
              dtype='datetime64[ns]', length=244, freq=None)

# DataFrame에 휴장일 적용하기

DataFrame에서 개장일(market day)만을 추출해해야 하는 경우를 살펴보자.

다음과 같은 데이터를 가정해 보자. (5월 1일~5월 30일, 1~30까지 valeus로 할당)

In [6]:
data = {'values': range(1,31)}
df_sample = pd.DataFrame(data, index=pd.date_range('2017-05-01', '2017-05-30'))
df_sample.head(10)

,values
2017-05-01,1
2017-05-02,2
2017-05-03,3
2017-05-04,4
2017-05-05,5
2017-05-06,6
2017-05-07,7
2017-05-08,8
2017-05-09,9
2017-05-10,10


위 데이터 중에서 개장일만 추출하고자 한다. DateTimeIndex를 가진 두 개의 DataFrame의 교집합을 구하는 간단한 방법으로 pd.merge()을 쓸 수 있다.

In [7]:
# 개장일을 index로 갖는 DataFrame
df_mdays = pd.DataFrame(index=mdays)

# 두 DataFrame (df_sample, df_mdays)의 인덱스를 기준으로 합친다(merge)
df = pd.merge(df_sample, df_mdays, right_index=True, left_index=True)
df.head(10)

,values
2017-05-02,2
2017-05-04,4
2017-05-08,8
2017-05-10,10
2017-05-11,11
2017-05-12,12
2017-05-15,15
2017-05-16,16
2017-05-17,17
2017-05-18,18


이 경우, 두 인덱스의 공동부분(intersection)만 남으므로 개장일만 남게 된다.

# CME, EUREX 휴장일

영업일 캘린더 메뉴에서 한국거래소(KRX)외에 CME, EUREX와 캘린더도 제공하며, 엑셀로 다운로드 한 2017년 데이터는 다음과 같다.

* hdays-2017-CME.xls ( https://goo.gl/cvnhy2 ) 
* hdays-2017-EUREX.xls ( https://goo.gl/s2PgBx ) 

In [8]:
pd.read_excel('https://goo.gl/cvnhy2')

,구분,일자,요일,시장,비고
0,휴장,2017/01/02,월,CME,New Year' Day
1,휴장,2017/04/14,금,CME,Good Friday
2,단축종료(03:15),2017/11/24,금,CME,추수감사절 다음날
3,휴장,2017/12/25,월,CME,Christmas Day


In [9]:
pd.read_excel('https://goo.gl/s2PgBx')

,구분,일자,요일,시장,비고
0,휴장,2017/04/14,금,EUREX,Good Friday
1,휴장,2017/04/17,월,EUREX,Easter Monday
2,휴장,2017/05/01,월,EUREX,Labour Day
3,휴장,2017/12/25,월,EUREX,Christmas Day
4,휴장,2017/12/26,화,EUREX,Boxing Day


# pandas-market-calendars

해외 시장의 개장일(market day)을 관리하기 위해 위 엑셀 데이터를 사용하기 보다는 pandas-market-calendars 라이브러를 권한다.

http://pandas-market-calendars.readthedocs.io

NYSE, CME, EUREX 등 시장에 대한 개장일과 개장 시간등을 관리할 수 있다.

#### 2017 https://fb.com/financedata